In [1]:
import geopandas as gpd
import pandas as pd
from geopandas import datasets, GeoDataFrame, read_file


%matplotlib inline

In [2]:
# reading the geopackage
cadastral_ref_gdf = gpd.read_file("Ciclica_test_gis.gpkg", layer='CadastralReferences')

C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [3]:
cadastral_ref_gdf.head()

,parcela_catastral,immb_us_prn,sup_total,sup_viv_sbr,demanda_cal_cte,geometry
0,63939911378595,Residencial,505.0,505.0,NaN,"MULTIPOLYGON (((595568.124 4417209.480, 595563..."
1,93428957957876,AltresNoCal,265.0,0.0,NaN,"MULTIPOLYGON (((595167.927 4419452.085, 595147..."
2,91748101647838,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((595171.157 4419445.925, 595171..."
3,38225472390951,Emmagatzematge,22.0,0.0,NaN,"MULTIPOLYGON (((595578.285 4416970.042, 595583..."
4,66150954127328,Emmagatzematge,9.0,0.0,NaN,"MULTIPOLYGON (((594968.813 4422211.527, 594966..."


In [4]:
# now we have a Geodataframe with valid geometry
type(cadastral_ref_gdf)

geopandas.geodataframe.GeoDataFrame

In [5]:
cadastral_ref_gdf.shape

(35991, 6)

In [6]:
cadastral_ref_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35991 entries, 0 to 35990
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   parcela_catastral  35991 non-null  object  
 1   immb_us_prn        35991 non-null  object  
 2   sup_total          35991 non-null  float64 
 3   sup_viv_sbr        35991 non-null  float64 
 4   demanda_cal_cte    23600 non-null  float64 
 5   geometry           35991 non-null  geometry
dtypes: float64(3), geometry(1), object(2)
memory usage: 1.6+ MB


In [7]:
cadastral_ref_gdf.describe() 

,sup_total,sup_viv_sbr,demanda_cal_cte
count,35991.000000,35991.000000,23600.000000
mean,425.627907,179.248159,6828.191080
std,1470.757165,373.147257,3876.109340
min,2.000000,0.000000,100.319108
25%,124.000000,54.000000,3460.362589
50%,213.000000,123.000000,6853.216876
75%,351.000000,198.000000,10179.524609
max,116159.000000,13389.000000,13499.576744


One time observed the main characteristics we will proceed with the exercices.

## 1. Assign census code id (‘id’ )  to every cadastral reference ( ‘parcela_catastral’).

As we don't have the id field we will have to obtain it form the data table CensusAreas.

In [8]:
census_ares_gdf = gpd.read_file('Ciclica_test_gis.gpkg', layer='CensusAreas')

C:\Users\34639\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [9]:
census_ares_gdf.head()

,id,geometry
0,0700201001,"MULTIPOLYGON (((4.19981 39.97058, 4.20001 39.9..."
1,0700202001,"MULTIPOLYGON (((4.13977 39.93362, 4.13977 39.9..."
2,0700202003,"MULTIPOLYGON (((4.13677 39.86414, 4.13657 39.8..."
3,0701501001,"MULTIPOLYGON (((3.84119 40.00544, 3.84063 40.0..."
4,0701501003,"MULTIPOLYGON (((3.83554 40.00015, 3.83341 39.9..."


In [10]:
census_ares_gdf.shape

(52, 2)

In [11]:
census_ares_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        52 non-null     object  
 1   geometry  52 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 960.0+ bytes


#### What to keep in mind?

Every cadastral_ref_gdf Multipolygon is contained in a census_area_gdf Multipolygon, and how there isn't a common field we have to do a **geospatial join** and for **one** census area witt be **many** cadastral_ref_gdf["parcela_catastral"]. The final table has to have the same rows as cadastral_ref_gdf-the null geometris(0).

[Spatial Joins Geopandas](https://geopandas.org/en/stable/gallery/spatial_joins.html)

https://geopandas.org/en/stable/docs/user_guide/mergingdata.html

[Own example](https://github.com/JosepAnSabate/jupyter_maps/blob/main/mapes_covid/6_mapacovid_vacunacio.ipynb)

In a LEFT OUTER JOIN (how='left'), we keep all rows from the left and duplicate them if necessary to represent multiple hits between the two dataframes. Spatial Joins Geopandas. 

In [14]:
from shapely.geometry import Point

In [15]:
# spatial join  between two GeoDataFrames
join_cadastral_gdf = census_ares_gdf.sjoin(cadastral_ref_gdf, census_ares_gdf,how="right", op="within")
join_cadastral_gdf

AttributeError: 'GeoDataFrame' object has no attribute 'sjoin'